In [1]:
''' 

Submitter : JESOON KANG, 20170937
Date : 2019. 10. 3


    Assignment 2. 

- Binary classification based on logistic regression -


'''
import matplotlib.pyplot as plt
import math
import numpy as np
import random

import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision
import os










In [2]:


# Image Data import & resize

transform = transforms.Compose([#transforms.Resize((256,256)),  
                                transforms.Grayscale(),		# the code transforms.Graysclae() is for changing the size [3,100,100] to [1, 100, 100] (notice : [channel, height, width] )
                                transforms.ToTensor(),])

#train_data_path = 'relative path of training data set'
train_data_path = './data/horse-or-human/train'
trainset = torchvision.datasets.ImageFolder(root=train_data_path, transform=transform)
# change the valuse of batch_size, num_workers for your program
# if shuffle=True, the data reshuffled at every epoch 
trainloader = torch.utils.data.DataLoader(trainset, batch_size=1, shuffle=False, num_workers=1)  


validation_data_path = './data/horse-or-human/validation'
valset = torchvision.datasets.ImageFolder(root=validation_data_path, transform=transform)
# change the valuse of batch_size, num_workers for your program
valloader = torch.utils.data.DataLoader(valset, batch_size=1, shuffle=False, num_workers=1)  







In [3]:
# sigmoid Function.
def sigmoid(z) :
    return 1 / (1 + math.e ** (-z))

In [4]:
# h(x) Function. hypothesis Func.
def hypothesis(weight,X) :
    theta = np.array(weight)
    z = np.dot(X,theta)
    h = sigmoid(z)
    return h

In [5]:
# Loss Function
def lossFunction(h,y) :
    # if i = 1, it occurs ZeroDivisionError ,
    # So, adjust value to 0.995
    for i in h :
        if i == 1 :
            i = 0.995
    try : 
        ret = (-y * np.log(h) - (1-y) * np.log(1-h)).mean()
    except ZeroDivisionError :
        print("Error")
    finally :
        return (-y * np.log(h) - (1-y) * np.log(1-h)).mean()

In [6]:
# predict Function. if h(x) returns >=0.5, set to 1. other cases, set to 0.
def predict(h, labels) :
    mount = len(h)
    correct = 0
    
    for i in range(0,mount) :
        if h[i] >= 0.5 :
            if labels[i] == 1 :
                correct +=1
        else :
            if labels[i] == 0 :
                correct +=1
    return correct * (1/mount)


In [13]:
# Data reconstruct. vectorize

# Each Image file will be stored shape of a row
training_vectorized = []
training_labels = []

validation_vectorized = []
validation_labels = []


# Training data vectorizing
for i, data in enumerate(trainloader) :
    train_data = []
    inputs, labels = data

    for u in inputs :
        for col in u[0] :
            train_data += list(col)
    training_vectorized.append(train_data)
    training_labels.append([labels])

training_vectorized = np.array(training_vectorized)
training_labels = np.array(training_labels)

# Validation data vectorizing
for i, data in enumerate(valloader) :
    val_data = []
    inputs, labels = data
    for u in inputs :
        for col in u[0] :
            val_data += list(col)
    validation_vectorized.append(val_data)
    validation_labels.append([labels])
validation_vectorized = np.array(validation_vectorized)
validation_labels = np.array(validation_labels)


In [14]:

# Initial Setting

# Initial Weight Value
weight = np.zeros((10000,1),dtype=float)

# Learning rate 
learning_rate = 0.00001

In [ ]:

# Log variables setting
log_training_loss = []
log_validation_loss = []
log_iter = []
log_training_acc = []
log_validation_acc = []


i = 0

log_training_loss.append(1)
# Training & Validation

while (True):
    i += 1
    h = hypothesis(weight,training_vectorized)
    
    gradient = np.dot(training_vectorized.T,h-training_labels) / len(training_vectorized)
    
    # adjust weight values with gradient value
    weight-= learning_rate*gradient    
    
    # get train loss value
    training_loss = lossFunction(h,training_labels)
    
    # get accuracy
    training_acc = predict(h,training_labels)
    
    
    h_validation = hypothesis(weight, validation_vectorized)
    validation_loss = lossFunction(h_validation, validation_labels)
    validation_acc = predict(h_validation, validation_labels)
    
    # add log data
    log_training_loss.append(training_loss)
    log_training_acc.append(training_acc)
    
    log_validation_loss.append(validation_loss)
    log_validation_acc.append(validation_acc)
    log_iter.append(i)
    
    #print("iter : ",i, "  t_loss : ",training_loss,"  t_acc : ", training_acc,"  v_loss: ",validation_loss, "  v_acc: ", validation_acc)
    #print(loss_log_train[-1])
    
    
    
    if ( (abs(training_loss - log_training_loss[-2]) < 0.0000001) ) :
        print("loss val is convergenced")
        break
    if (i == 10000) :
        print("EPOCH reached 10000. terminate process")
        break
    if (i%100 == 0) :
        print("Current num_epoch : ",i)
    
    
    
print("Finished")    


In [ ]:
log_training_loss[0].remov
t1 = plt.plot(log_iter,log_training_loss, color='orange',label='Training Loss')
t2 = plt.plot(log_iter,log_validation_loss, color= 'red',label='Validation Loss')
plt.title("Loss")
plt.legend(['Training Loss','Validation Loss'])
plt.show()

In [ ]:
t1 = plt.plot(log_iter,log_training_acc, color='orange',label='Training Acc')
t2 = plt.plot(log_iter,log_validation_acc, color= 'red',label='Validation Acc')
plt.title("Accuracy")
plt.legend(['Training Acc','Validation Acc'])
plt.show()

In [ ]:
# commit Point.